# Flucht aus dem Labyrinth

**Aufgabe:** Schreibe ein Programm, das einen Weg durch das folgende Labyrinth findet, vom Eingang in der obersten Zeile zum Ausgang in der untersten Zeile.

In [10]:
labyrinth01 = """
########### #########
# #               # #
# # ### ######### # #
#     #         # # #
# ############# # # #
# # #           # # #
# # # ########### # #
#   #   #     #   # #
##### # # # ### ### #
#   # # # # #   #   #
# # ### # # # ##### #
# #   # # # # #   # #
##### # # ### # # # #
#   #   #   #   #   #
### ##### # ####### #
#     #   #       # #
# ##### ##### ##### #
#     # #   #       #
# ### # ### #########
#   #               #
########### #########
"""

Weitere zufällige Labyrinthe kannst du hier erzeugen:
https://thenerdshow.com/amaze.html?rows=10&cols=10  
(Markiere das Labyrinth und füge es als String ein.)

### Bewegen im Labyrinth
Bevor wir beginnen können, einen kompletten Weg durchs Labyrinth zu suchen, überlegen wir zuerst, wie man einzelne Schritte beschreiben kann.

Eine Position schreiben wir als 2-Tupel (Paar), z.B.

```
pos = (11, 0)  # die Koordinaten des Eingangs beim obigen Labyrinth
```

Es gibt vier "Züge", d.h. Bewegungsmöglichkeiten: links, rechts, hoch und runter.
Jede Bewegung entspricht einer Positionsänderung in x- oder y-Richtung.

 Dies könnte man durch mehrere `if`-Statements (in anderen Sprachen: `switch`) lösen:
```
px, py = pos   # "tuple unpacking" (Destrukturierung), d.h. Datenstruktur auf mehrere Einzelvariablen verteilen
if zug == "rechts":
    px += 1
elif zug == "hoch":
    py += -1
elif zug == "links":
    ...   # usw.
```

Das wird schnell unübersichtlich und ist auch konzeptuell unschön: Eigentlich ist doch
jedem String zug ein bestimmer Richtungsvektor `(dx, dy)` zugeordnet. Wir verwenden also besser 
eine Art "Lookup-Table" bzw. in Python ein **Dictionary**:

In [30]:
zuege = dict(links=(-1, 0), rechts=(1, 0), hoch=(0, -1), runter=(0, 1))   # zuege["links"] == (-1, 0) usw.

Damit können wir nun eine einfach Funktion definieren, die Züge (Bewegungen) im Labyrinth durchführt.

*Hinweis: Wie beim Schiebepuzzle verzichten wir an dieser Stelle auf die Überprüfung, ob der Zug überhaupt erlaubt ist. Dies Auswahl "legaler" Züge treffen wir später und können **hier** deshalb darauf verzichten.*

In [12]:
def gehe(pos, richtung):
    px, py = pos   # "tuple unpacking"
    dx, dy = zuege[richtung]
    return px+dx, py+dy

# Bsp:
gehe((11, 0), "runter")

(11, 1)

Nun müssen wir noch beschreiben, welche Bewegungen an einer bestimmten Position im Labyrinth erlaubt sind. Dazu prüfen wir für jeden der möglichen Züge, ob seine Ausführung auf ein leeres Feld des Labyrinths führt.

Dafür können wir praktischerweise die gerade programmierte Funktion `gehe()` verwenden.

In [29]:
def labyrinth_einlesen(lab_str):
    return [zeile.rstrip('\n') for zeile in lab_str.splitlines() if zeile.startswith("#")]

def ist_frei(pos, labyrinth):
    px, py = pos    
    hoehe = len(labyrinth)
    breite = len(labyrinth[0])
    return 0 <= px < breite and 0 <= py < hoehe and labyrinth[py][px] == ' '

def moegliche_bewegungsrichtungen(pos, labyrinth):
    px, py = pos
    erlaubte = [richtung for richtung in zuege if ist_frei(gehe(pos, richtung), labyrinth)]
    return erlaubte

# Bsp:
moegliche_bewegungsrichtungen((11, 1), labyrinth_einlesen(labyrinth01))


['links', 'rechts', 'hoch']

Nun haben wir alles, was wir benötigen, um die Wegsuche in Labyrinthen als *Suchproblem* zu beschreiben!

### Das Labyrinth als Suchproblem


In [ ]:
from breitensuche import Suchproblem

# Es gibt vier "Züge", d.h. Bewegungsmöglichkeiten: links, rechts, hoch und runter.
# Jede Bewegung entspricht einer Positionsänderung in x- und y-Richtung.
# Die Zuordnung zug --> (deltax, deltay) speichern wir in einem Dictionary:
zuege = dict(links=(-1, 0), rechts=(1, 0), hoch=(0, -1), runter=(0, 1))

class Labyrinth(Suchproblem):

    def __init__(self, lab):
        """lädt das Labyrinth aus eine Multiline-String ein und findet Start- und Zielpunkt."""
        self.labyrinth = [zeile.rstrip('\n') for zeile in lab.splitlines() if zeile.startswith("#")]
        self.hoehe = len(self.labyrinth)
        self.breite = len(self.labyrinth[0])
        self.startzustand = (self.labyrinth[0].index(' '), 0)  # Leerzeichen in Zeile 0
        self.zielzustand = (self.labyrinth[-1].index(' '), self.hoehe - 1)  # Leerzeichen in letzter Zeile

    def fuehre_aktion_aus(self, zustand, aktion):
        """Bsp. laby.fuehre_aktion_aus((11,0), "runter")  --> (11, 1)"""
        # TODO: hier vervollständigen

    def moegliche_aktionen(self, zustand):
        """laby.moegliche_aktionen((11, 1))  -->  ['links', 'rechts', 'hoch']"""
        # TODO: hier vervollständigen

    def darstellen(self, loesung=None, markierung='.'):
        if loesung is None:
            loesung = []
        lab = list(self.labyrinth)  # Kopie des Labyrinths
        weg = [self.startzustand]
        for zug in loesung:
            pos = weg[-1]  # letzte Position
            pos_neu = self.fuehre_aktion_aus(pos, zug)
            weg.append(pos_neu)
        for x, y in weg:
            lab[y] = lab[y][:x] + markierung + lab[y][x + 1:]  # Weg durch Markierung ersetzen
        for zeile in lab:
            print(zeile)

In [ ]:
problem = Labyrinth(labyrinth01)
problem.darstellen()

###########.#########
# #               # #
# # ### ######### # #
#     #         # # #
# ############# # # #
# # #           # # #
# # # ########### # #
#   #   #     #   # #
##### # # # ### ### #
#   # # # # #   #   #
# # ### # # # ##### #
# #   # # # # #   # #
##### # # ### # # # #
#   #   #   #   #   #
### ##### # ####### #
#     #   #       # #
# ##### ##### ##### #
#     # #   #       #
# ### # ### #########
#   #               #
########### #########


In [ ]:
problem.startzustand

(11, 0)

In [ ]:
problem.fuehre_aktion_aus(problem.startzustand, "runter")

(11, 1)

In [ ]:
problem.moegliche_aktionen((11, 1))

['links', 'rechts', 'hoch']

In [ ]:
from breitensuche import breitensuche_graph
loesung = breitensuche_graph(problem)
print(loesung)
print("Weglänge:", len(loesung))

['runter', 'rechts', 'rechts', 'rechts', 'rechts', 'rechts', 'rechts', 'runter', 'runter', 'runter', 'runter', 'runter', 'runter', 'links', 'links', 'runter', 'runter', 'links', 'links', 'runter', 'runter', 'runter', 'runter', 'rechts', 'rechts', 'hoch', 'hoch', 'rechts', 'rechts', 'runter', 'runter', 'rechts', 'rechts', 'runter', 'runter', 'runter', 'runter', 'links', 'links', 'links', 'links', 'links', 'links', 'hoch', 'hoch', 'links', 'links', 'hoch', 'hoch', 'links', 'links', 'runter', 'runter', 'links', 'links', 'runter', 'runter', 'runter', 'runter', 'rechts', 'rechts', 'rechts', 'rechts', 'runter']
Weglänge: 64


In [ ]:
problem.darstellen(loesung)

###########.#########
# #        .......# #
# # ### #########.# #
#     #         #.# #
# ############# #.# #
# # #           #.# #
# # # ###########.# #
#   #   #     #...# #
##### # # # ###.### #
#   # # # # #...#   #
# # ### # # #.##### #
# #   # # # #.#...# #
##### # # ###.#.#.# #
#   #   #...#...#...#
### #####.#.#######.#
#     #...#...    #.#
# #####.#####.#####.#
#     #.#   #.......#
# ### #.### #########
#   #  .....        #
###########.#########
